In [20]:
from nwpackageversions import StatusChecker, StatusSummary

requirements_fp : str = r"/workspaces/nwpackageversions/docs/ExampleFiles/requirements.txt"
devcontainer_fp : str = r"/workspaces/nwpackageversions/docs/ExampleFiles/Dockerfile"

status_checker : StatusChecker = StatusChecker()
status_checker.try_check(file_path = requirements_fp)


[Package(name='requests', version='2.26.0'), Package(name='beautifulsoup4', version='4.10.0'), Package(name='asyncio', version='3.4.3'), Package(name='lxml', version='4.6.3'), Package(name='typed-astunparse', version='2.1.4'), Package(name='tensorflow-hub', version='0.12.0'), Package(name='Django', version='3.1.5'), Package(name='pytz', version='2022.1'), Package(name='line-profiler', version='1.0'), Package(name='opencv-python', version='4.10.0.84'), Package(name='black', version='22.12.0'), Package(name='certifi', version='2022.12.7'), Package(name='chardet', version='5.1.0'), Package(name='charset-normalizer', version='3.1.0')]
['FROM python:3.12.5-bookworm', '', '', 'RUN wget https://github.com/jgm/pandoc/releases/download/3.4/pandoc-3.4-1-amd64.deb \\', '    && dpkg -i pandoc-3.4-1-amd64.deb \\', '    && rm -f pandoc-3.4-1-amd64.deb']
